In [ ]:
# Import Python libraries
import ipywidgets as widgets
from IPython.display import Javascript, display
from time import sleep

In [ ]:
%%js
// 'Import Javascript libraries'
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});
require(['phidget22'], (phidget22) => {
   window.phidget22 = phidget22; 
});

In [ ]:
## We pause for a second here, to allow the library to load
sleep(1)

Make the USB connection between the Phidget and the browser:

In [ ]:
%%js

if (window.usbconn === undefined) {
    
    element.text("Creating a new USB Connection.");
    
    window.usbconn = new phidget22.USBConnection();
    
    usbconn.connect().then(() => {
        usbconn.requestWebUSBDeviceAccess();
    }).catch(err => {
        window.usbconn.delete();
        element.append("Error connecting to USB" + err);
    });
}

In [ ]:
## We rest for a bit while the USB port opens
sleep(1)


In [ ]:
%%js
// 'Check for USB connection'
element.text(usbconn.connected);

Connect Phidgets to notebook:

In [ ]:
%%js

// 'Soil moisture sensor'
window.soilSensor = new phidget22.VoltageRatioInput();
soilSensor.open();

// 'Water pump'
window.pump = new phidget22.DigitalOutput();
pump.setIsHubPortDevice(true);
pump.open();

// 'Light sensor'
window.lightSensor = new phidget22.LightSensor();
lightSensor.open();

In [ ]:
# Turn pump on
Javascript('pump.setState(true);')

In [ ]:
%%js
// 'Turn pump off, this time in Javascript'
pump.setState(false);

In [ ]:
# Light sensor
for i in range(10):
    display(Javascript("element.text(lightSensor.getIlluminance());"))
    sleep(.25)

In [ ]:
# Moisture sensor
for i in range(10):
    display(Javascript("element.text(soilSensor.getVoltageRatio());"))
    sleep(.25)

In [ ]:
def doDisconnect(b):
    display(Javascript("""
        (async () => {
            await humSensor.close(); usbconn.close(); usbconn.delete(); delete window.usbconn;
            element.text("You have disconnected the Phidgets.");
            })();
    """))

run_button = widgets.Button(
    description = 'IMPORTANT: Click to disconnect', 
        button_style='danger',layout=widgets.Layout(width='50%', height='80px')
)
print("Press this button when you are done, to disconnect the Phidgets")
run_button.on_click(doDisconnect)

display(run_button)